In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import ipywidgets as widgets
from IPython.display import display, clear_output

# -----------------------------
# Step 1: Download gold price data
# -----------------------------
gold_data = yf.download("GC=F", start="2015-01-01", end="2026-12-31")
df = gold_data[['Close']].copy().dropna()

# -----------------------------
# Step 2: Normalize data
# -----------------------------
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# -----------------------------
# Step 3: Create sequences
# -----------------------------
window_size = 10
X, y = [], []
for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i-window_size:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

# -----------------------------
# Step 4: Train/Test Split
# -----------------------------
split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# -----------------------------
# Step 5: Build and Train LSTM
# -----------------------------
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X_train.shape[1],1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

# -----------------------------
# Step 6: Date picker and output
# -----------------------------
date_picker = widgets.DatePicker(
    description='Pick a date',
    value=pd.to_datetime("2024-01-01").date(),
    min=pd.to_datetime("2015-01-01").date(),
    max=pd.to_datetime("2025-09-11").date()
)
display(date_picker)

output = widgets.Output()
display(output)

# -----------------------------
# Step 7: Prediction function for next 5 days
# -----------------------------
def predict_next_days(change, days=5):
    with output:
        clear_output()
        try:
            # Convert date to Timestamp
            date_input = pd.Timestamp(date_picker.value)
            
            # Pick closest previous trading day
            valid_dates = df.index[df.index <= date_input]
            if len(valid_dates) == 0:
                closest_date = df.index[-1]
            else:
                closest_date = valid_dates[-1]

            # Get last 'window_size' days
            idx = df.index.get_loc(closest_date)
            if idx < window_size:
                print("Not enough data before this date for prediction.")
                return
            
            last_sequence = scaled_data[idx-window_size:idx].reshape(1, window_size, 1)
            predictions = []

            current_sequence = last_sequence.copy()
            for _ in range(days):
                next_scaled = model.predict(current_sequence)
                next_price = scaler.inverse_transform(next_scaled)[0][0]
                predictions.append(next_price)

                # Update sequence: remove first day, append new prediction
                next_scaled_reshaped = next_scaled.reshape(1,1,1)
                current_sequence = np.append(current_sequence[:,1:,:], next_scaled_reshaped, axis=1)
            
            print(f"Using date: {closest_date.date()}")
            print(f"Closing Price: {df.loc[closest_date, 'Close']:.2f} USD")
            for i, price in enumerate(predictions, 1):
                print(f"Predicted day {i}: {price:.2f} USD")
            
            # Plot predictions
            plt.figure(figsize=(10,5))
            plt.plot(range(1, days+1), predictions, marker='o', linestyle='-')
            plt.title(f"Next {days} Day Gold Price Forecast")
            plt.xlabel("Day")
            plt.ylabel("Price (USD)")
            plt.grid(True)
            plt.show()
            
        except Exception as e:
            print("Error:", e)

# Link the function to the date picker
date_picker.observe(lambda change: predict_next_days(change, days=5), names='value')

def predict_next_days(change, days=5):
    with output:
        clear_output()
        try:
            # Convert date to Timestamp
            date_input = pd.Timestamp(date_picker.value)
            
            # Pick closest previous trading day
            valid_dates = df.index[df.index <= date_input]
            if len(valid_dates) == 0:
                closest_date = df.index[-1]
            else:
                closest_date = valid_dates[-1]

            # Get last 'window_size' days
            idx = df.index.get_loc(closest_date)
            if idx < window_size:
                print("Not enough data before this date for prediction.")
                return
            
            last_sequence = scaled_data[idx-window_size:idx].reshape(1, window_size, 1)
            predictions = []

            current_sequence = last_sequence.copy()
            for _ in range(days):
                next_scaled = model.predict(current_sequence)
                next_price = scaler.inverse_transform(next_scaled)[0][0]
                predictions.append(next_price)

                # Update sequence: remove first day, append new prediction
                next_scaled_reshaped = next_scaled.reshape(1,1,1)
                current_sequence = np.append(current_sequence[:,1:,:], next_scaled_reshaped, axis=1)
            
            # Use scalar value for printing
            closing_price = df.loc[closest_date, 'Close'].iloc[0] if isinstance(df.loc[closest_date, 'Close'], pd.Series) else df.loc[closest_date, 'Close']

            print(f"Using date: {closest_date.date()}")
            print(f"Closing Price: {closing_price:.2f} USD")
            for i, price in enumerate(predictions, 1):
                print(f"Predicted day {i}: {price:.2f} USD")
            
            # Plot predictions
            plt.figure(figsize=(10,5))
            plt.plot(range(1, days+1), predictions, marker='o', linestyle='-')
            plt.title(f"Next {days} Day Gold Price Forecast")
            plt.xlabel("Day")
            plt.ylabel("Price (USD)")
            plt.grid(True)
            plt.show()
            
        except Exception as e:
            print("Error:", e)




C:\Users\HP\AppData\Local\Temp\ipykernel_13080\3446678563.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  gold_data = yf.download("GC=F", start="2015-01-01", end="2026-12-31")
[*********************100%***********************]  1 of 1 completed


Epoch 1/20
67/67 [==============================] - 2s 4ms/step - loss: 0.0040
Epoch 2/20
67/67 [==============================] - 0s 3ms/step - loss: 1.2952e-04
Epoch 3/20
67/67 [==============================] - 0s 3ms/step - loss: 1.2245e-04
Epoch 4/20
67/67 [==============================] - 0s 3ms/step - loss: 1.2053e-04
Epoch 5/20
67/67 [==============================] - 0s 3ms/step - loss: 1.1758e-04
Epoch 6/20
67/67 [==============================] - 0s 3ms/step - loss: 1.1450e-04
Epoch 7/20
67/67 [==============================] - 0s 3ms/step - loss: 1.1434e-04
Epoch 8/20
67/67 [==============================] - 0s 3ms/step - loss: 1.1041e-04
Epoch 9/20
67/67 [==============================] - 0s 4ms/step - loss: 1.0851e-04
Epoch 10/20
67/67 [==============================] - 0s 3ms/step - loss: 1.0738e-04
Epoch 11/20
67/67 [==============================] - 0s 4ms/step - loss: 1.0547e-04
Epoch 12/20
67/67 [==============================] - 0s 4ms/step - loss: 1.0247e-04
Epoch

DatePicker(value=datetime.date(2024, 1, 1), description='Pick a date', max=datetime.date(2025, 9, 11), min=dat…

Output()